In [ ]:
# LLM module
class LLM():
    def IPT2NTL(self, IPT, NTL):
        # IPT to NTL code needed
        return IPT, NTL
    
    def OUTPUT(self, IPTs, NTL, K): # IPTs 는 리스트
        out = 'output_val' # output_val 연산 코드 필요
        return out

In [ ]:
# 감정 저장소
class emotion_storage():
    def __init__(self):
        self.NTL_bef = [] # 이전 상태의 NTL
        self.K = []
    
    def NTL2NTLandK(self, NTL):
        K = -70 # 초기 전위
        w = 0.1 # 이전 NTL 적용 가중치(10%)
        if len(K) != 0:
            for i, j in enumerate(self.K): # 시간 감쇠를 적용한 K 업데이트
                K += i**(1/j)
        NTL_bef_w = [i*w for i in self.NTL_bef]
        NTL_final = [NTL[i] + NTL_bef_w[i] for i in range(4)]
        return NTL_final, K
    
    def NTL_K_hist(self, NTL, K): # 마지막 모델에서 계산한 NTL, K를 현재 스텝의 최종 NTL, K로 기록
        self.NTL_bef = NTL
        self.K.append(K)

In [ ]:
# 뉴런 정의
K_remember = 100 # 기억 막전위

class neuron():
    def __init__(self, K_critical, code, type): # 노드 고유 번호(code)
        self.K_critical = K_critical # 임계 막전위(활성화를 위한)
        self.saved_IPT = [None]
        self.code = code
        self.connected_node = [[nodes[((self.code+5)%len(nodes))+1]]]
        self.connected_node_code = [((self.code+5)%len(nodes))+1]
        self.w_bef = 0
        self.w_curr = 0
        self.K_out = []
        self.NTL_out = []
        self.IPT_curr = ''

        self.type = type # types: exciting, inhibiting, modulating

        self.K_final = 0
        self.NTL_final = 0

    def do_work(self, NTL, K):
        #w 업데이트 시에는 self.w_curr을 업데이트 하기!
        # 할일 code needed
        return NTL, K
    
    def save(self, IPT):
        self.saved_IPT.append(IPT)
        self.IPT_curr = IPT

    def make_new_connection(self, node): # 노드와 연결 함수
        self.connected_node.append(node)
        node.connected_node.append(self)

        self.connected_node_code.append(node.code)
        node.connected_node_code.append(self.code)
        return node.code

    def main(self, NTL, K, IPT, node):
        if K > self.K_critical:
            NTL_out, K_out = self.do_work(NTL, K)
            self.NTL_out.append(NTL_out)
            if node.code != 0:
                self.K_out.append(K_out * node.w_bef) # 각 노드 별로 계산된 (K_out * 해당 노드의 가중치)형태로 저장
            else:
                self.K_out.append(K_out) # 초기 뉴런 일때

        if K > K_remember: # IPT 기억 조건
            self.save(IPT)
            print('[LOG] IPT Saved')

        if self.w_curr - self.w_bef > 0: # 가중치가 커지면 그만크 연결 노드 수 증가
            for _ in range(int(self.w_curr - self.w_bef)):
                available_node = list(set(node_codes + self.connected_node_code))
                self.make_new_connection(nodes[available_node[0]-1])

        K_final = sum(K_out)
        NTL_final = []
        for i in range(4):
            for j in NTL_out:
                k += j[i]
            NTL_final.append(k/len(NTL_out))
            k=0

        self.NTL_final = NTL_final
        self.K_final = K_final
        self.update_nodes()
        return
    
    def update_nodes(self):
        a_kept = self.saved_IPT.copy()                # 원본 복사본
        IPT_out = a_kept[:a_kept.index(self.IPT_curr)] + a_kept[a_kept.index(self.IPT_curr)+1:]
        for i in self.connected_node:
            i.main(self.NTL_final, self.K_final, IPT_out[-1], self) # saved_IPT[-1] 은 현재 

In [ ]:
# nn 정의
w_exciting = 0.33
w_inhibiting = 0.33
w_modulating = 0.34
node_count = 100
node_codes = []
nodes = []

for i in range(int(node_count*w_exciting)):
    nodes.append(neuron(+30, i, 'exciting'))
for i in range(int(node_count*w_inhibiting)):
    nodes.append(neuron(+30, i, 'inhibiting'))
for i in range(int(node_count*w_modulating)):
    nodes.append(neuron(+30, i, 'modulating'))
node_count = len(nodes)

**FLOW**

In [ ]:
# class들 호출
LLM = LLM()
emotion_storage = emotion_storage()

In [ ]:
# 데이터 박스 구성
data_box = dict()

In [ ]:
# INPUT
data_box['IPT'] = 'I hate you'

In [ ]:
# 감정 저장소
data_box['NTL'] = LLM.IPT2NTL(data_box['IPT'])
data_box['NTL'], data_box['K'] = emotion_storage.NTL2NTLandK(data_box['NTL'])

In [ ]:
# 초기 뉴런 자극
import random

class dummy_node():
    def __init__(self):
        self.code = 0

dummy_node = dummy_node()

start_neuron = nodes[random.randrange(1, node_count+1)+1]
start_neuron.main(data_box['NTL'], data_box['K'], data_box['IPT'], dummy_node)
